## Задание 6.1

In [1]:
## Импорт библиотек

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn.pipeline
import sklearn.feature_extraction
import sklearn.svm
import sklearn.model_selection
import re as re
import sklearn.metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

In [3]:
positive = pd.read_csv('positive.csv',sep=';',error_bad_lines=False, header=None).iloc[:,[3,4]]
negative = pd.read_csv('negative.csv',sep=';',error_bad_lines=False, header=None).iloc[:,[3,4]]

In [4]:
# проверка на сбалансированность
print(positive.shape,negative.shape)

(114911, 2) (111923, 2)


In [5]:
## объединим позитивные и негативные вместе
inp=pd.concat([positive,negative])

In [6]:
## удалим упоминания других участников беседы
inp[3] = inp[3].map(lambda x:re.sub(r'[R][T]\s+[@]\w+[:]\s*|[@]\w+\s+','', x))

In [7]:
inp.head(5)

,3,4
0,"хоть я и школота, но поверь, у нас то же самое...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,Ну ты идиотка) я испугалась за тебя!!!,1
3,"""Кто то в углу сидит и погибает от голода, а м...",1
4,"Вот что значит страшилка :D\nНо блин,посмотрев...",1


In [8]:
x_train, x_test, y_train, y_test = train_test_split(inp[3], inp[4], test_size=0.1)

In [9]:
x_train.shape

(204150,)

In [10]:
x_test.shape

(22684,)

In [11]:
grid_search = sklearn.model_selection.GridSearchCV(
    sklearn.pipeline.Pipeline([('vectorize', None),('classifier', sklearn.svm.LinearSVC())]),
    param_grid={
        'vectorize':[
            ##  default max_df=1.0 min_df=1.0
            
            sklearn.feature_extraction.text.CountVectorizer(),
            
            ## min_df=5 слово должно встречаться минимум в 5 докумнтах
            ## max_df=0.9 уберем слова которые встречаются более чем в 90 процентах докуметов
            
            sklearn.feature_extraction.text.CountVectorizer(max_df=0.9, min_df=5),
            
            sklearn.feature_extraction.text.TfidfVectorizer(),
            
             ## min_df=5 слово должно встречаться минимум в 5 докумнтах
            ## max_df=0.9 уберем слова которые встречаются более чем в 90 процентах докуметов
            sklearn.feature_extraction.text.TfidfVectorizer(max_df=0.9, min_df=5),
        ]
    }
)

In [12]:
%%time
grid_search.fit(x_train, y_train)

Wall time: 6min 11s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vectorize', None), ('classifier', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'vectorize': [CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
   ...
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [13]:
grid_search.cv_results_

{'mean_fit_time': array([ 47.4159685 ,  40.81866908,   6.57856464,   6.37035108]),
 'mean_score_time': array([ 1.66429766,  1.4934477 ,  1.56945491,  1.51709692]),
 'mean_test_score': array([ 0.72557923,  0.72765124,  0.74047514,  0.73260838]),
 'mean_train_score': array([ 0.96113642,  0.82197404,  0.93733039,  0.81710997]),
 'param_vectorize': masked_array(data = [ CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 1), preprocessor=None, stop_words=None,
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None, vocabulary=None)
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.9, max_features=None, min_df=5,
         ngram_range=(1, 1), preprocesso

In [14]:
    predict=grid_search.best_estimator_.predict(x_test)

In [15]:
print(sklearn.metrics.classification_report(y_test, predict))

             precision    recall  f1-score   support

         -1       0.75      0.75      0.75     11198
          1       0.75      0.75      0.75     11486

avg / total       0.75      0.75      0.75     22684



In [16]:
sklearn.metrics.confusion_matrix(y_test,predict)

array([[8363, 2835],
       [2832, 8654]])